In [1]:
import warnings
warnings.filterwarnings(action='ignore')

## Install libraries

In [3]:
# !pip install -q youtube-transcript-api langchain-community langchain-openai faiss-cpu tiktoken python-dotenv

In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

## Step 1a - Indexing (Document Ingestion)

In [ ]:
# Video Link : https://youtu.be/Gfr50f6ZBvo?si=rjJLdo8Smux4G-CZ

video_id = "Gfr50f6ZBvo"  # only the ID, not full URL

try:
    # Instantiate the API class
    ytt = YouTubeTranscriptApi()
    # Fetch the transcript (prioritizes English; use a list for languages even if single)
    transcript_list = ytt.fetch(video_id, languages=["en"])
    # Flatten it to plain text (transcript_list is iterable over dicts with 'text' keys)
    transcript = " ".join(chunk.text for chunk in transcript_list)
    print(f"Transcript : {transcript}")
except TranscriptsDisabled:
    print("No captions available for this video.")
except NoTranscriptFound:
    print("No English transcript found; try other languages or check if subtitles exist.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
transcript_list

## Step 1b - Indexing (Text Splitting)

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [ ]:
len(chunks), chunks[0]

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)

In [ ]:
vector_store.index_to_docstore_id

In [ ]:
vector_store.get_by_ids(['a68b54f4-5476-4706-b735-a998b9591790'])

## Step 2 - Retrieval

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 4, "score_threshold":0.0})
retriever

In [ ]:
# Example
retriever.invoke('What is deepmind')

## Step 3 - Augmentation

In [ ]:
question = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"

In [ ]:
retrieved_docs = retriever.invoke(question)
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.
      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

## Step 4 - Generation

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

## Building a Chain

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('is the topic of nuclear fusion discussed in this video? if yes then what was discussed')

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('Can you summarize the video')